In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 11:41:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from pathlib import Path

import pandas as pd

In [3]:
from pyspark.sql import types

In [19]:
yellow_schema = types.StructType(
    [
        types.StructField("VendorID", types.IntegerType(), True),
        types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
        types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
        types.StructField("passenger_count", types.IntegerType(), True),
        types.StructField("trip_distance", types.FloatType(), True),
        types.StructField("RatecodeID", types.IntegerType(), True),
        types.StructField("store_and_fwd_flag", types.StringType(), True),
        types.StructField("PULocationID", types.IntegerType(), True),
        types.StructField("DOLocationID", types.IntegerType(), True),
        types.StructField("payment_type", types.IntegerType(), True),
        types.StructField("fare_amount", types.FloatType(), True),
        types.StructField("extra", types.FloatType(), True),
        types.StructField("mta_tax", types.FloatType(), True),
        types.StructField("tip_amount", types.FloatType(), True),
        types.StructField("tolls_amount", types.FloatType(), True),
        types.StructField("improvement_surcharge", types.FloatType(), True),
        types.StructField("total_amount", types.FloatType(), True),
        types.StructField("congestion_surcharge", types.FloatType(), True),
    ]
)

green_schema = types.StructType(
    [
        types.StructField("VendorID", types.IntegerType(), True),
        types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
        types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
        types.StructField("store_and_fwd_flag", types.StringType(), True),
        types.StructField("RatecodeID", types.IntegerType(), True),
        types.StructField("PULocationID", types.IntegerType(), True),
        types.StructField("DOLocationID", types.IntegerType(), True),
        types.StructField("passenger_count", types.IntegerType(), True),
        types.StructField("trip_distance", types.FloatType(), True),
        types.StructField("fare_amount", types.FloatType(), True),
        types.StructField("extra", types.FloatType(), True),
        types.StructField("mta_tax", types.FloatType(), True),
        types.StructField("tip_amount", types.FloatType(), True),
        types.StructField("tolls_amount", types.FloatType(), True),
        types.StructField("ehail_fee", types.FloatType(), True),
        types.StructField("improvement_surcharge", types.FloatType(), True),
        types.StructField("total_amount", types.FloatType(), True),
        types.StructField("payment_type", types.IntegerType(), True),
        types.StructField("trip_type", types.IntegerType(), True),
        types.StructField("congestion_surcharge", types.FloatType(), True),
    ]
)

In [ ]:
df_yellow = pd.read_csv(
    "data/raw/yellow/2021/01/yellow_tripdata_2021-01.csv.gz", nrows=1000
)
df_green = pd.read_csv(
    "data/raw/green/2021/01/green_tripdata_2021-01.csv.gz", nrows=1000
)

In [97]:
services = {"yellow": yellow_schema, "green": green_schema}
for service, schema in services.items():
    for input_path in Path(f"data/raw/{service}").rglob("*.csv.gz"):
        output_path = Path("data/pq") / input_path.parent.relative_to("data/raw")
        output_path.parent.mkdir(parents=True, exist_ok=True)

        df = (
            spark.read.option("header", "true")
            .schema(schema)
            .csv(str(input_path.parent))
        )

        df.repartition(4).write.parquet(str(output_path))